In [2]:
import heapq as h
import numpy as np

POISSON_PROCESS_NEW_REQUESTS_LAMBDA = 1
PARETO_PROCESS_NEW_FILE_SIZE_A = 1
PARETO_PROCESS_FILE_POPULARITY_A = 1
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN = 0
LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA = 1

TOTAL_NO_OF_FILES = 10000
INSTITUTIONAL_BANDWIDTH = 100
ACCESS_LINK_BANDWIDTH = 15
TOTAL_TIME_TO_RUN = 10
CACHE_CAPACITY = 1000


class Files:
    def __init__(self):
        self.popularity = np.random.pareto(PARETO_PROCESS_FILE_POPULARITY_A,size=TOTAL_NO_OF_FILES)
        self.popularity = self.popularity/np.sum(self.popularity)
        self.size = np.random.pareto(PARETO_PROCESS_NEW_FILE_SIZE_A,size=TOTAL_NO_OF_FILES)



class Simulation_Q():
    def __init__(self):
        self.q = []
    
    def push(self, event_tuple:tuple):
        h.heappush(self.q,event_tuple)

    def pop(self):
        return h.heappop(self.q)[1]
        
class Cache():
    def __init__(self,
    all_files:Files,
    init_file_list=[]):

        self.capacity = CACHE_CAPACITY
        self.store = {}
        self.storage_left = CACHE_CAPACITY
        self.all_files = all_files

        for i in range(len(init_file_list)):
            if not self.__add_file__(init_file_list[i]):
                break

    def __add_file__(self,file_index):
        if file_index not in self.store:
            if self.storage_left > self.all_files.size[file_index]:
                self.store[file_index] = self.all_files.size[file_index]
                self.storage_left -= self.all_files.size[file_index]
            else:
                return False
        return True
            



class Simulator_Env():
    def __init__(self,
    cache_init_files=[]):

        self.sim_q = Simulation_Q()
        self.files = Files()
        self.cache = Cache(self.files,cache_init_files)
        self.fifo = []
        self.log = []
        self.req_count = 0


class Event:

    def __init__(self, sim: Simulator_Env, create_time: int, parent:object=None):
        self.sim = sim
        self.create_time = create_time
        self.process_time = create_time
        self.parent = parent
        self.name = 'Event'
        self.__enqueue__()

    def get_super_parent(self):
        node = self
        while(node.parent is not None):
            node = node.parent
        return node

    def __lt__(self,any):
        return True

    def __gt__(self,any):
        return True

    def __enqueue__(self):
        pass


    def process(self):
        pass


class E_get_new_reqs(Event):

    def __enqueue__(self):
        self.name = 'Get New Requests'
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        reqs_to_handle = np.random.poisson(POISSON_PROCESS_NEW_REQUESTS_LAMBDA)
        for i in range(int(reqs_to_handle)):
            E_new_req(self.sim, self.process_time)


class E_new_req(Event):
    def __enqueue__(self):
        self.name = 'New Request'
        self.sim.req_count += 1
        self.file_index = np.argmax(np.random.multinomial(1,self.sim.files.popularity))
        self.file_size = self.sim.files.size[self.file_index]        
        # self.process_time = self.create_time + (self.file_size/INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if self.file_index in sim.cache.store:
            E_file_recieved(self.sim,self.process_time,self)
        else:
            E_arrive_at_queue(self.sim,self.process_time,self)


class E_file_recieved(Event):
    def __enqueue__(self):
        self.name = 'File recieved'
        initial_req = self.get_super_parent()
        file_size = self.sim.files.size[initial_req.file_index]
        self.process_time = self.create_time + (file_size/ INSTITUTIONAL_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        log_data = [self.process_time - initial_req.create_time, initial_req.file_index, initial_req.file_size,self]
        self.sim.log.append(log_data)


class E_arrive_at_queue(Event):
    def __enqueue__(self):
        self.name = 'Arrive at queue'
        self.process_time = self.create_time + np.random.lognormal(LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_MEAN,
                                                                    LOGNORMAL_PROCESS_ARRIVE_AT_QUEUE_SIGMA)
        
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        if len(self.sim.fifo):
            self.sim.fifo.append(self)
        else:
            E_depart_from_queue(self.sim,self.process_time,self)


class E_depart_from_queue(Event):
    def __enqueue__(self):
        self.name = 'Depart from queue'
        initial_req = self.get_super_parent()
        self.process_time = self.create_time + (initial_req.file_size/ACCESS_LINK_BANDWIDTH)
        self.sim.sim_q.push([self.process_time,self])

    def process(self):
        initial_req = self.get_super_parent()
        sim.cache.__add_file__(initial_req.file_index)
        E_file_recieved(self.sim,self.process_time,self)
        if len(sim.fifo):
            event = sim.fifo.pop(0)
            E_depart_from_queue(self.sim,self.process_time,event)











In [5]:
# initialize simulator environment
np.random.seed(11)
sim = Simulator_Env()

#initialize new req events to be processed at every second
for i in range(TOTAL_TIME_TO_RUN):
    E_get_new_reqs(sim,i)


#Main simulator loop
while(len(sim.sim_q.q)):

    e = sim.sim_q.pop()
    e.process()

print(len(sim.log) == sim.req_count)

#Show logs
for data in sim.log:    
    e = data[3]
    path = []
    while(e is not None):
        path.insert(0,e.name)
        e = e.parent
    print('file name - ' + str(data[1]))    
    print('file size - ' + str(data[2]))
    print('total time - ' + str(data[0]))
    print(path)
    print(' ')


True
file name - 1786
file size - 3.157538702430733
total time - 0.652675789333257
['New Request', 'Arrive at queue', 'Depart from queue', 'File recieved']
 
file name - 4099
file size - 1.038420293642159
total time - 0.9288550861052995
['New Request', 'Arrive at queue', 'Depart from queue', 'File recieved']
 
file name - 4099
file size - 1.038420293642159
total time - 0.010384202936421616
['New Request', 'File recieved']
 
file name - 4445
file size - 0.08126625219887074
total time - 0.29916469675959445
['New Request', 'Arrive at queue', 'Depart from queue', 'File recieved']
 
file name - 4099
file size - 1.038420293642159
total time - 0.010384202936421616
['New Request', 'File recieved']
 
file name - 9801
file size - 0.8095794391049609
total time - 0.12562601583180477
['New Request', 'Arrive at queue', 'Depart from queue', 'File recieved']
 
file name - 5271
file size - 4.99580699988733
total time - 0.8460506651787174
['New Request', 'Arrive at queue', 'Depart from queue', 'File rec